In [1]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 

llm = ChatOpenAI(
    model="gpt-5-nano-2025-08-07", 
    temperature=0, 
    openai_api_key=OPENAI_API_KEY
)

#### Initialize DuckDuckGo Search Tool

In [2]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
search_tool

DuckDuckGoSearchRun(api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='auto', source='text'))

#### Initialize OpenWeatherMap Tool

In [3]:
import os
from langchain_community.utilities import OpenWeatherMapAPIWrapper

# `load_tools` is not available in your installed LangChain; build the tool directly instead.
from langchain_core.tools import Tool

api_key = os.getenv("OPENWEATHERMAP_API_KEY")

weather = OpenWeatherMapAPIWrapper(openweathermap_api_key=api_key)
weather_tool = Tool.from_function(func=weather.run, name="openweathermap", description="Get current weather from OpenWeatherMap.") if hasattr(Tool, "from_function") else Tool(name="openweathermap", func=weather.run, description="Get current weather from OpenWeatherMap.")

weather_tool

Tool(name='openweathermap', description='Get current weather from OpenWeatherMap.', func=<bound method OpenWeatherMapAPIWrapper.run of OpenWeatherMapAPIWrapper(owm=<pyowm.owm.OWM - API key=************************6e9677e4, subscription type=free, PyOWM version=('3', '5', '0')>, openweathermap_api_key='6a241a44864d653ff36c211c6e9677e4')>)

In [23]:
tools = [weather_tool, search_tool]

##### Create Built-in Agent

In [ ]:
from langchain.agents import create_agent

# Defining the system prompt as a string explicitly
system_prompt = """You are a helpful assistant that can use tools when needed. 
                   When you use a tool, respond with an action and action input. 
                   Return the final answer when you are done."""

# Creating the agent using create_agent
agent = create_agent(
    model=llm, 
    tools=tools, 
    system_prompt=system_prompt
)

response = agent.invoke(
                {"messages": [{"role": "user", "content": "what is the weather in Kandy?"}]}
            )
print(response)

{'messages': [HumanMessage(content='what is the weather in Kandy?', additional_kwargs={}, response_metadata={}, id='5ce1625b-d071-43c7-a6e3-7688a27c8029'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 349, 'prompt_tokens': 233, 'total_tokens': 582, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D2WJOPqWnxcXc7meXRLsIvhV5cU6k', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bfe03-fc5f-70a0-93aa-b2b3e3293c59-0', tool_calls=[{'name': 'openweathermap', 'args': {'__arg1': 'Kandy, Sri Lanka'}, 'id': 'call_qRRoop5DzHkUYhk5AWjFMFT1', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 233,

In [19]:
response = agent.invoke(
                {"messages": [{"role": "user", "content": "what is ML?"}]}
            )

In [20]:
print(response)

{'messages': [HumanMessage(content='what is ML?', additional_kwargs={}, response_metadata={}, id='65895655-4e5a-4c6b-9e64-64b2b5c66871'), AIMessage(content='ML stands for machine learning. It’s a subfield of artificial intelligence focused on building systems that learn from data, rather than being explicitly programmed for every task.\n\nKey ideas:\n- Instead of writing explicit rules, you provide data and a model that learns patterns from that data.\n- The model makes predictions or decisions on new, unseen data.\n\nCommon types:\n- Supervised learning: learn from labeled examples (input features x, target y). Tasks include:\n  - Regression (predict a continuous value, e.g., house prices)\n  - Classification (predict a category, e.g., spam vs. not spam)\n- Unsupervised learning: learn from unlabeled data to find structure or patterns. Tasks include:\n  - Clustering (group similar items)\n  - Dimensionality reduction (reduce the number of features)\n  - Anomaly detection\n- Reinforcem

#### Define custom tool

In [22]:
from langchain.tools import tool

@tool
def word_count_tool(text: str) -> int:
    """Counts the number of words in the given text."""
    return str(len(text.split()))

In [24]:
tools.append(word_count_tool)

print(tools)

[Tool(name='openweathermap', description='Get current weather from OpenWeatherMap.', func=<bound method OpenWeatherMapAPIWrapper.run of OpenWeatherMapAPIWrapper(owm=<pyowm.owm.OWM - API key=************************6e9677e4, subscription type=free, PyOWM version=('3', '5', '0')>, openweathermap_api_key='6a241a44864d653ff36c211c6e9677e4')>), DuckDuckGoSearchRun(api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='auto', source='text')), StructuredTool(name='word_count_tool', description='Counts the number of words in the given text.', args_schema=<class 'langchain_core.utils.pydantic.word_count_tool'>, func=<function word_count_tool at 0x0000024B8CD71300>)]


In [25]:
agent = create_agent(
    model=llm, 
    tools=tools, 
    system_prompt=system_prompt
)

response = agent.invoke(
                {"messages": [{"role": "user", "content": "How many words are in introduction to machine learning?"}]}
            )
print(response)

{'messages': [HumanMessage(content='How many words are in introduction to machine learning?', additional_kwargs={}, response_metadata={}, id='011455b5-26bb-4edf-90ca-1fac05e4bae5'), AIMessage(content='If you’re referring to the phrase itself, “Introduction to machine learning” has 4 words.\n\nIf you mean a specific document (e.g., a book chapter or article), please share the text or a link, and I can count the words for you.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 830, 'prompt_tokens': 260, 'total_tokens': 1090, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 768, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D2WcbR2DBzpy2CVyl0bKLB3buztST', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='l